# Caso de Negocio

### [BBT3-9](https://alkemy-labs.atlassian.net/browse/BBT3-9) Fabricio

### [BBT3-10](https://alkemy-labs.atlassian.net/browse/BBT3-10) Judith

### [BBT3-11](https://alkemy-labs.atlassian.net/browse/BBT3-11) Pablo

### Creacion notebook e importación de librerías.

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=e2922e5cf6986b3a4868ea03245e58efa0224840a06c60d3eb954ff8be2c38a1
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
import pandas as pd
from pyspark.sql import SparkSession
import sqlite3 as sql
import os

## Datasets a base de datos

### [BBT3-12](https://alkemy-labs.atlassian.net/browse/BBT3-12) Fabricio

### [BBT3-13](https://alkemy-labs.atlassian.net/browse/BBT3-13) Judith

### [BBT3-14](https://alkemy-labs.atlassian.net/browse/BBT3-14) Pablo

In [3]:
order_items_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_order_items_dataset.csv'
customers_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_customers_dataset.csv'
geolocation_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_geolocation_dataset.csv'
sellers_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_sellers_dataset.csv'
order_payments_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_order_payments_dataset.csv'
product_category_name_translation_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20product_category_name_translation.csv'
order_reviews_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_order_reviews_dataset.csv'
orders_link_csv = 'https://github.com/alkemyTech/BBVA-BDS-W1-T3/raw/main/datasets/Copia%20de%20ecommerce_orders_dataset.csv'
products_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_products_dataset.csv'


order_items_df = pd.read_csv(order_items_link_csv)
order_payments_df = pd.read_csv(order_payments_link_csv)
products_df = pd.read_csv(products_link_csv)
customers_df = pd.read_csv(customers_link_csv)
orders_df = pd.read_csv(orders_link_csv)
order_reviews_df = pd.read_csv(order_reviews_link_csv)
geolocation_df = pd.read_csv(geolocation_link_csv)
sellers_df = pd.read_csv(sellers_link_csv)
product_category_name_translation_df = pd.read_csv(product_category_name_translation_link_csv)

In [4]:
# seteo de columnas indices:
order_items_df.set_index(['order_id', 'order_item_id'], inplace=True)
order_payments_df.set_index(['order_id','payment_sequential'], inplace=True)
products_df.set_index('product_id', inplace=True)
customers_df.set_index('customer_id', inplace=True)
orders_df.set_index('order_id', inplace=True)
order_reviews_df.set_index('review_id', inplace=True)
geolocation_df.set_index('geolocation_zip_code_prefix', inplace=True)
sellers_df.set_index('seller_id', inplace=True)
product_category_name_translation_df.set_index('product_category_name', inplace=True)

In [5]:
# creacion base de datos
conn = sql.connect('ecommerce.db')
cursor = conn.cursor()

In [6]:
# guardado de dataframes en tablas de la base de datos
order_payments_df.to_sql('order_payments', con=conn, if_exists='replace')
order_items_df.to_sql('order_items', con=conn, if_exists='replace')
orders_df.to_sql('orders', con=conn, if_exists='replace')
customers_df.to_sql('customers', con=conn, if_exists='replace')
products_df.to_sql('products', con=conn, if_exists='replace')
order_reviews_df.to_sql('order_reviews', con=conn, if_exists='replace')
geolocation_df.to_sql('geolocation', con=conn, if_exists='replace')
sellers_df.to_sql('sellers', con=conn, if_exists='replace')
product_category_name_translation_df.to_sql('product_category_name_translation', con=conn, if_exists='replace')

71

In [7]:
# query para obtener los nombres de las tablas creadas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
lista_tablas = cursor.fetchall()
lista_nombre_tablas = [tabla[0] for tabla in lista_tablas]
print(lista_nombre_tablas)

['order_payments', 'order_items', 'orders', 'customers', 'products', 'order_reviews', 'geolocation', 'sellers', 'product_category_name_translation']


## Extraccion de Base de Datos


### orders_items y sellers
###[BBT3-17](https://alkemy-labs.atlassian.net/browse/BBT3-17) Judith

In [8]:
query_items_seller = '''
  SELECT * FROM order_items
  INNER JOIN sellers
  ON order_items.seller_id = sellers.seller_id
'''
items_seller_df = pd.read_sql_query(query_items_seller, conn)
items_seller_df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,48436dade18ac8b2bce089ec2a041202,27277,volta redonda,SP
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,dd7ddc04e1b6c2c614352b383efe2d36,3471,sao paulo,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,5b51032eddd242adc84c38acab88f23d,37564,borda da mata,MG
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,9d7a1d34a5052409006425275ba1c2b4,14403,franca,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,df560393f3a51e74553ab94004ba5c87,87900,loanda,PR


### orders_items y products
### [BBT3-18](https://alkemy-labs.atlassian.net/browse/BBT3-18) Judith

In [9]:
query_items_products = '''
  SELECT * FROM order_items
  INNER JOIN products
  ON order_items.product_id = products.product_id
'''
items_products_df = pd.read_sql_query(query_items_products, conn)
items_products_df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,e5f2d52b802189ee658865ca93d83a8f,pet_shop,56.0,239.0,2.0,30000.0,50.0,30.0,40.0
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,c777355d18b72b67abbeef9df44fd0fd,moveis_decoracao,59.0,695.0,2.0,3050.0,33.0,13.0,33.0
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,7634da152a4610f1595efa32f14722fc,perfumaria,42.0,480.0,1.0,200.0,16.0,10.0,15.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,ac6c3623068f30de03045865e4e10089,ferramentas_jardim,59.0,409.0,1.0,3750.0,35.0,40.0,30.0


### orders y order_payments
### [BBT3-19](https://alkemy-labs.atlassian.net/browse/BBT3-19) Judith

In [10]:
query_order_payments = '''
  SELECT * FROM orders
  INNER JOIN order_payments
  ON orders.order_id = order_payments.order_id
'''
query_order_payments = pd.read_sql_query(query_order_payments, conn)
query_order_payments.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,0a8556ac6be836b46b3e89920d59291c,delivered,2018-04-25 22:01:49,2018-04-25 22:15:09,2018-05-02 15:20:00,2018-05-09 17:36:51,2018-05-22 00:00:00,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,f2c7fc58a9de810828715166c672f10a,delivered,2018-06-26 11:01:38,2018-06-26 11:18:58,2018-06-28 14:18:00,2018-06-29 20:32:09,2018-07-16 00:00:00,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,25b14b69de0b6e184ae6fe2755e478f9,delivered,2017-12-12 11:19:55,2017-12-14 09:52:34,2017-12-15 20:13:22,2017-12-18 17:24:41,2018-01-04 00:00:00,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,7a5d8efaaa1081f800628c30d2b0728f,delivered,2017-12-06 12:04:06,2017-12-06 12:13:20,2017-12-07 20:28:28,2017-12-21 01:35:51,2018-01-04 00:00:00,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,15fd6fb8f8312dbb4674e4518d6fa3b3,delivered,2018-05-21 13:59:17,2018-05-21 16:14:41,2018-05-22 11:46:00,2018-06-01 21:44:53,2018-06-13 00:00:00,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


### Extracción de datos desde las tablas customers y orders
### [BBT3-15](https://alkemy-labs.atlassian.net/browse/BBT3-15) - Fabricio

In [11]:
query_customers_orders = '''
SELECT *
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
'''
customers_and_orders_df = pd.read_sql_query(query_customers_orders, conn)

In [12]:
customers_and_orders_df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,9ef432eb6251297304e76186b10a928d,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,b0830fb4747a6c6d20dea0b8c802d7ef,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,41ce2a54c0b03bf3443c3d931a367089,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,f88197465ea7920adcdbec7375364d82,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,8ab97904e6daea8866dbdbc4fb7aad2c,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


### Extracción de datos desde las tablas orders y orders_items
### [BBT3-16](https://alkemy-labs.atlassian.net/browse/BBT3-16) - Fabricio

In [13]:
query_orders_and_order_items = '''
SELECT *
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
'''
orders_and_order_items_df = pd.read_sql_query(query_orders_and_order_items, conn)

In [14]:
orders_and_order_items_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


### Extracción de datos desde las tablas orders, payments y customers
### [BBT3-24](https://alkemy-labs.atlassian.net/browse/BBT3-24) - Fabricio

In [15]:
query_orders_payments_and_customers = '''
SELECT *
FROM orders o
JOIN order_payments op ON o.order_id = op.order_id
JOIN customers c ON o.customer_id = c.customer_id
'''
orders_payments_and_customers_df = pd.read_sql_query(query_orders_payments_and_customers, conn)

In [16]:
orders_payments_and_customers_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_id,payment_sequential,payment_type,payment_installments,payment_value,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,b81ef226f3fe1789b1e8b2acac839d17,0a8556ac6be836b46b3e89920d59291c,delivered,2018-04-25 22:01:49,2018-04-25 22:15:09,2018-05-02 15:20:00,2018-05-09 17:36:51,2018-05-22 00:00:00,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33,0a8556ac6be836b46b3e89920d59291c,708ab75d2a007f0564aedd11139c7708,39801,teofilo otoni,MG
1,a9810da82917af2d9aefd1278f1dcfa0,f2c7fc58a9de810828715166c672f10a,delivered,2018-06-26 11:01:38,2018-06-26 11:18:58,2018-06-28 14:18:00,2018-06-29 20:32:09,2018-07-16 00:00:00,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39,f2c7fc58a9de810828715166c672f10a,a8b9d3a27068454b1c98cc67d4e31e6f,2422,sao paulo,SP
2,25e8ea4e93396b6fa0d3dd708e76c1bd,25b14b69de0b6e184ae6fe2755e478f9,delivered,2017-12-12 11:19:55,2017-12-14 09:52:34,2017-12-15 20:13:22,2017-12-18 17:24:41,2018-01-04 00:00:00,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71,25b14b69de0b6e184ae6fe2755e478f9,6f70c0b2f7552832ba46eb57b1c5651e,2652,sao paulo,SP
3,ba78997921bbcdc1373bb41e913ab953,7a5d8efaaa1081f800628c30d2b0728f,delivered,2017-12-06 12:04:06,2017-12-06 12:13:20,2017-12-07 20:28:28,2017-12-21 01:35:51,2018-01-04 00:00:00,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78,7a5d8efaaa1081f800628c30d2b0728f,87695ed086ebd36f20404c82d20fca87,36060,juiz de fora,MG
4,42fdf880ba16b47b59251dd489d4441a,15fd6fb8f8312dbb4674e4518d6fa3b3,delivered,2018-05-21 13:59:17,2018-05-21 16:14:41,2018-05-22 11:46:00,2018-06-01 21:44:53,2018-06-13 00:00:00,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45,15fd6fb8f8312dbb4674e4518d6fa3b3,4291db0da71914754618cd789aebcd56,18570,conchas,SP


### Extracción de datos desde las tablas orders y reviews

### [BBT3-22](https://alkemy-labs.atlassian.net/browse/BBT3-22) Pablo


Crear un DataFrame que contenga el JOIN de la tabla orders y tabla reviews.

In [17]:
query_orders_reviews = '''
SELECT *
FROM orders AS o
INNER JOIN order_reviews AS orv
  ON o.order_id = orv.order_id
;
'''
orders_reviews_df = pd.read_sql_query(query_orders_reviews, conn)
orders_reviews_df.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,73fc7af87114b39712e6da79b0a377eb,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,2018-01-11 15:47:59,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,None,None,2018-01-18 00:00:00,2018-01-18 21:46:59
1,a548910a1c6147796b98fdf73dbeba33,8a2e7ef9053dea531e4dc76bd6d853e6,delivered,2018-02-28 12:25:19,2018-02-28 12:48:39,2018-03-02 19:08:15,2018-03-09 23:17:20,2018-03-14 00:00:00,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,None,None,2018-03-10 00:00:00,2018-03-11 03:05:13
2,f9e4b658b201a9f2ecdecbb34bed034b,e226dfed6544df5b7b87a48208690feb,delivered,2018-02-03 09:56:22,2018-02-03 10:33:41,2018-02-06 16:18:28,2018-02-16 17:28:48,2018-03-09 00:00:00,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,None,None,2018-02-17 00:00:00,2018-02-18 14:36:24


### Extracción de datos desde las tablas orders_items, products y products_category_translation

### [BBT3-23](https://alkemy-labs.atlassian.net/browse/BBT3-23) Pablo


Crear un DataFrame que contenga el JOIN de la tabla orders_items, tabla products y product_category_name_translation.  

In [18]:
query_ordersitems_products = '''
SELECT *
FROM order_items AS oi
INNER JOIN products AS p
  ON oi.product_id = p.product_id
INNER JOIN product_category_name_translation AS pct
  ON p.product_category_name = pct.product_category_name
;
'''
orderitems_products_df = pd.read_sql_query(query_ordersitems_products, conn)
orderitems_products_df.head(3)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name,product_category_name_english
0,e17e4f88e31525f7deef66779844ddce,1,1e9e8ef04dbcff4541ed26657ea517e5,5670f4db5b62c43d542e1b2d56b0cf7c,2018-04-30 17:33:54,10.91,7.39,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumaria,perfumery
1,5236307716393b7114b53ee991f36956,1,3aa071139cb16b67ca9e5dea641aaa2f,b561927807645834b59ef0d16ba55a24,2018-02-06 19:11:15,248.00,17.99,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,artes,art
2,01f66e58769f84129811d43eefd187fb,1,96bd76ec8810374ed1b65e291975717f,7b07b3c7487f0ea825fc6df75abd658b,2018-07-11 21:30:20,79.80,7.82,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,esporte_lazer,sports_leisure


### Extracción de datos desde las tablas customers y geolocations

### [BBT3-20](https://alkemy-labs.atlassian.net/browse/BBT3-20) Pablo

Crear un DataFrame que contenga el JOIN de la tabla customers y tabla geolocation.

In [19]:
# Hacemos el promedio de coordenadas por cada zip_code y calculamos tambien la desviacion standard de esas
# coordenadas para evitar conflictos en futuras tareas
# Desviacion standard calculada como: std = (avg(col*col) - avg(col)*avg(col))^(1/2)
# siendo std^2 la varianza

query_geolocation_customers = '''
SELECT *
FROM (
  SELECT geolocation_zip_code_prefix, AVG(geolocation_lat) AS mean_lat, AVG(geolocation_lng) as mean_lng,
          SQRT(avg(geolocation_lat * geolocation_lat) - avg(geolocation_lat)*avg(geolocation_lat)) AS std_lat,
          SQRT(avg(geolocation_lng * geolocation_lng) - avg(geolocation_lng)*avg(geolocation_lng)) AS std_lng
  FROM geolocation
  GROUP BY geolocation_zip_code_prefix
  ) AS geoavg
INNER JOIN customers AS c
  ON c.customer_zip_code_prefix = geoavg.geolocation_zip_code_prefix
;
'''

geolocation_customer_df = pd.read_sql_query(query_geolocation_customers, conn)
geolocation_customer_df.head(3)

,geolocation_zip_code_prefix,mean_lat,mean_lng,std_lat,std_lng,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,14409,-20.498489,-47.396929,0.013494,0.005581,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,9790,-23.727992,-46.542848,0.007551,0.009140,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,1151,-23.531642,-46.656289,0.001380,0.001577,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP


### Extracción de datos desde las tablas orders y geolocations

### [BBT3-21](https://alkemy-labs.atlassian.net/browse/BBT3-21) Pablo

Crear un DataFrame que contenga el JOIN de la tabla orders y tabla geolocation.  

In [20]:
# Hacemos el promedio de coordenadas por cada zip_code y calculamos tambien la desviacion standard de esas
# coordenadas para evitar conflictos en futuras tareas
# Desviacion standard calculada como: std = (avg(col*col) - avg(col)*avg(col))^(1/2)
# siendo std^2 la varianza

query_geolocation_orders = '''
SELECT *
FROM (
  SELECT geolocation_zip_code_prefix, AVG(geolocation_lat) AS mean_lat, AVG(geolocation_lng) as mean_lng,
          SQRT(avg(geolocation_lat * geolocation_lat) - avg(geolocation_lat)*avg(geolocation_lat)) AS std_lat,
          SQRT(avg(geolocation_lng * geolocation_lng) - avg(geolocation_lng)*avg(geolocation_lng)) AS std_lng
  FROM geolocation
  GROUP BY geolocation_zip_code_prefix
  ) AS geoavg
INNER JOIN customers AS c
  ON c.customer_zip_code_prefix = geoavg.geolocation_zip_code_prefix
INNER JOIN orders AS o
  ON c.customer_id = o.customer_id
;
'''

geolocation_orders_df = pd.read_sql_query(query_geolocation_orders, conn)
geolocation_orders_df.head(3)

,geolocation_zip_code_prefix,mean_lat,mean_lng,std_lat,std_lng,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,3149,-23.576983,-46.587161,0.002764,0.000383,9ef432eb6251297304e76186b10a928d,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,47813,-12.177924,-44.660711,0.181920,1.462450,b0830fb4747a6c6d20dea0b8c802d7ef,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,75265,-16.745150,-48.514783,0.003059,0.005126,41ce2a54c0b03bf3443c3d931a367089,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00


## Filtrar datos de DataFrame

### orders y customers
### [BBT3-25](https://alkemy-labs.atlassian.net/browse/BBT3-25) Judith

In [21]:
#Elimine la columna duplicada customer_id
customers_and_orders_df = customers_and_orders_df.loc[:, ~customers_and_orders_df.columns.duplicated(keep='first')]
customers_and_orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   customer_id                    99441 non-null  object
 1   customer_unique_id             99441 non-null  object
 2   customer_zip_code_prefix       99441 non-null  int64 
 3   customer_city                  99441 non-null  object
 4   customer_state                 99441 non-null  object
 5   order_id                       99441 non-null  object
 6   order_status                   99441 non-null  object
 7   order_purchase_timestamp       99441 non-null  object
 8   order_approved_at              99281 non-null  object
 9   order_delivered_carrier_date   97658 non-null  object
 10  order_delivered_customer_date  96476 non-null  object
 11  order_estimated_delivery_date  99441 non-null  object
dtypes: int64(1), object(11)
memory usage: 9.1+ MB


In [22]:
#Aqui agrupo los datos por customer_unique_id, y me quedo con los ultimos 3 ordenes de compras ingresadas a la Base de Datos
customers_and_orders_filtrado_df = customers_and_orders_df.groupby('customer_unique_id').tail(3)
customers_and_orders_filtrado_df.count()

customer_id                      99345
customer_unique_id               99345
customer_zip_code_prefix         99345
customer_city                    99345
customer_state                   99345
order_id                         99345
order_status                     99345
order_purchase_timestamp         99345
order_approved_at                99185
order_delivered_carrier_date     97563
order_delivered_customer_date    96382
order_estimated_delivery_date    99345
dtype: int64

In [23]:
#Estoy verificando que ningun customer_unique_id se repite mas de 3 veces
verificacion = customers_and_orders_filtrado_df.groupby('customer_unique_id').size().reset_index(name='conteo')
verificacion.query('conteo > 3')

,customer_unique_id,conteo


### orders y orders_items
### [BBT3-26](https://alkemy-labs.atlassian.net/browse/BBT3-26) Judith

In [24]:
#Elimine la columna duplicada order_id
orders_and_order_items_df = orders_and_order_items_df.loc[:, ~orders_and_order_items_df.columns.duplicated(keep='first')]
orders_and_order_items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 14 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       112650 non-null  object 
 1   customer_id                    112650 non-null  object 
 2   order_status                   112650 non-null  object 
 3   order_purchase_timestamp       112650 non-null  object 
 4   order_approved_at              112635 non-null  object 
 5   order_delivered_carrier_date   111456 non-null  object 
 6   order_delivered_customer_date  110196 non-null  object 
 7   order_estimated_delivery_date  112650 non-null  object 
 8   order_item_id                  112650 non-null  int64  
 9   product_id                     112650 non-null  object 
 10  seller_id                      112650 non-null  object 
 11  shipping_limit_date            112650 non-null  object 
 12  price                         

In [25]:
#Aqui ordeno los precios de manera ascendente y agrupo las filas por order_id de manera de tener todos los items juntos de una misma orden
#por lo tanto al estar de manera ascendente me quedo con los ultimos 3 items de mayor precio por orden de compra
orders_and_order_items_df = orders_and_order_items_df.sort_values(by='price', ascending=True)
orders_and_order_items_filtrado_df = orders_and_order_items_df.groupby('order_id').tail(3)
orders_and_order_items_filtrado_df.count()

order_id                         110756
customer_id                      110756
order_status                     110756
order_purchase_timestamp         110756
order_approved_at                110741
order_delivered_carrier_date     109608
order_delivered_customer_date    108355
order_estimated_delivery_date    110756
order_item_id                    110756
product_id                       110756
seller_id                        110756
shipping_limit_date              110756
price                            110756
freight_value                    110756
dtype: int64

In [26]:
#Verificacion de que no haya mas de 3 items por cliente
orders_and_order_items_filtrado_df.groupby('order_id')['order_item_id'].count().sort_values()

order_id
00010242fe8c5a6d1ba2dd792cb16214    1
a6e9d106235bcf1dda54253686d89e99    1
a6e9b80a7636eb8dd592dbb3e20d0a91    1
a6e963c11e80432334e984ead4797a8b    1
a6e8ad5db31e71f5f12671af561acb4a    1
                                   ..
8171523911786efd1d91c66d69051fcd    3
cf0e7f7b96ea1d2939d084ae7b99f10a    3
a1e3ae78b2bf6dedf2c6a37f96e17900    3
cf46c4a1a3f077bf5cd7e232b9bfb796    3
b5f13aa3a46ca3005d39389331866fe4    3
Name: order_item_id, Length: 98666, dtype: int64

### orders_items y sellers
### [BBT3-27](https://alkemy-labs.atlassian.net/browse/BBT3-27) Judith

In [27]:
#Elimine la columna duplicada seller_id
items_seller_df = items_seller_df.loc[:, ~items_seller_df.columns.duplicated(keep='first')]
items_seller_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   order_id                112650 non-null  object 
 1   order_item_id           112650 non-null  int64  
 2   product_id              112650 non-null  object 
 3   seller_id               112650 non-null  object 
 4   shipping_limit_date     112650 non-null  object 
 5   price                   112650 non-null  float64
 6   freight_value           112650 non-null  float64
 7   seller_zip_code_prefix  112650 non-null  int64  
 8   seller_city             112650 non-null  object 
 9   seller_state            112650 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 8.6+ MB


In [28]:
#Aqui ordeno los precios de manera ascendente y agrupo las filas por order_id de manera de tener todos los items juntos de una misma orden
#por lo tanto al estar de manera ascendente me quedo con los ultimos 3 items de mayor precio por vendedor
items_seller_df = items_seller_df.sort_values(by='price', ascending=True)
items_seller_filtrado_df = items_seller_df.groupby('order_id').tail(3)
items_seller_filtrado_df.count()

order_id                  110756
order_item_id             110756
product_id                110756
seller_id                 110756
shipping_limit_date       110756
price                     110756
freight_value             110756
seller_zip_code_prefix    110756
seller_city               110756
seller_state              110756
dtype: int64

In [29]:
#Verificacion de que no haya mas de 3 items por vendedor
items_seller_filtrado_df.groupby('order_id')['order_item_id'].count().sort_values()

order_id
00010242fe8c5a6d1ba2dd792cb16214    1
a6e9d106235bcf1dda54253686d89e99    1
a6e9b80a7636eb8dd592dbb3e20d0a91    1
a6e963c11e80432334e984ead4797a8b    1
a6e8ad5db31e71f5f12671af561acb4a    1
                                   ..
8171523911786efd1d91c66d69051fcd    3
cf0e7f7b96ea1d2939d084ae7b99f10a    3
a1e3ae78b2bf6dedf2c6a37f96e17900    3
cf46c4a1a3f077bf5cd7e232b9bfb796    3
b5f13aa3a46ca3005d39389331866fe4    3
Name: order_item_id, Length: 98666, dtype: int64

### orders_items y products
### [BBT3-28](https://alkemy-labs.atlassian.net/browse/BBT3-28) Judith

In [30]:
#Elimine la columna duplicada product_id
items_products_df = items_products_df.loc[:, ~items_products_df.columns.duplicated(keep='first')]
items_products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 15 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   order_id                    112650 non-null  object 
 1   order_item_id               112650 non-null  int64  
 2   product_id                  112650 non-null  object 
 3   seller_id                   112650 non-null  object 
 4   shipping_limit_date         112650 non-null  object 
 5   price                       112650 non-null  float64
 6   freight_value               112650 non-null  float64
 7   product_category_name       111047 non-null  object 
 8   product_name_lenght         111047 non-null  float64
 9   product_description_lenght  111047 non-null  float64
 10  product_photos_qty          111047 non-null  float64
 11  product_weight_g            112632 non-null  float64
 12  product_length_cm           112632 non-null  float64
 13  product_height

In [31]:
#Aqui ordeno los precios de manera ascendente y agrupo las filas por order_id de manera de tener todos los items juntos de una misma orden
#por lo tanto al estar de manera ascendente me quedo con los ultimos 3 items de mayor precio por categoria
items_products_df = items_products_df.sort_values(by='price', ascending=True)
items_products_filtrado_df = items_products_df.groupby('order_id').tail(3)
items_products_filtrado_df.count()

order_id                      110756
order_item_id                 110756
product_id                    110756
seller_id                     110756
shipping_limit_date           110756
price                         110756
freight_value                 110756
product_category_name         109174
product_name_lenght           109174
product_description_lenght    109174
product_photos_qty            109174
product_weight_g              110738
product_length_cm             110738
product_height_cm             110738
product_width_cm              110738
dtype: int64

In [32]:
#Verificacion de que no haya mas de 3 items por categoria
items_seller_filtrado_df.groupby('order_id')['order_item_id'].count().sort_values()

order_id
00010242fe8c5a6d1ba2dd792cb16214    1
a6e9d106235bcf1dda54253686d89e99    1
a6e9b80a7636eb8dd592dbb3e20d0a91    1
a6e963c11e80432334e984ead4797a8b    1
a6e8ad5db31e71f5f12671af561acb4a    1
                                   ..
8171523911786efd1d91c66d69051fcd    3
cf0e7f7b96ea1d2939d084ae7b99f10a    3
a1e3ae78b2bf6dedf2c6a37f96e17900    3
cf46c4a1a3f077bf5cd7e232b9bfb796    3
b5f13aa3a46ca3005d39389331866fe4    3
Name: order_item_id, Length: 98666, dtype: int64

### Reducir volumen de order y payments
### [BBT3-29](https://alkemy-labs.atlassian.net/browse/BBT3-29) Fabricio

Filtrar el DataFrame de órdenes y pagos para resguardar los 3 pagos de mayor importe por cliente.  

In [33]:
# Hacer un CTE (Common Table Expression) para hacer como una subconsulta y optimizar los tiempos
# Hacer un ranking en base al numero de fila (Row_number()) Y luego tomar aquellos que row_number desde el 1 al 3

query_bbt3_29 = '''
WITH RankedPayments AS (
  SELECT
    o.order_id,
    c.customer_unique_id,
    p.payment_value,
    ROW_NUMBER() OVER (PARTITION BY c.customer_unique_id ORDER BY p.payment_value DESC) as rank
  FROM
    orders AS o
  JOIN order_payments p ON o.order_id = p.order_id
  JOIN customers c ON o.customer_id = c.customer_id
)
SELECT
  order_id,
  customer_unique_id,
  payment_value
FROM
  RankedPayments
WHERE 1=1
  AND rank <= 3
ORDER BY
  payment_value DESC
'''

top_3_payments_bcostumers = pd.read_sql_query(query_bbt3_29, conn)

In [34]:
# Corroborar que cada cliente posee solo 3 registros
top_3_payments_bcostumers.groupby('customer_unique_id')['payment_value'].count().sort_values()

customer_unique_id
0000366f3b9a7992bf8c76cfdf3221e2    1
a7b78c748cd03d589d69ac3230526fc0    1
a7b781410bcc8bbf3221f48ff45aae6d    1
a7b692898d3828ce34e905a4356faa05    1
a7b5f908c796ad17ce9339238ef15a87    1
                                   ..
8bd6a7f99463f04ac042cc25d3450405    3
97bc08e526795b17e1d4f642f77ae304    3
c7c19110042036e6266adc998cbf1289    3
8be158f05b793cd72d5fde01365b5d2c    3
e016365b624d1755f3e2d0e8f1631ebb    3
Name: payment_value, Length: 96095, dtype: int64

In [35]:
# Ejemplo de un customer id que tiene 3 valores, corroborar que estén cargados de mayor a menor
top_3_payments_bcostumers[top_3_payments_bcostumers['customer_unique_id'] == 'e016365b624d1755f3e2d0e8f1631ebb']

,order_id,customer_unique_id,payment_value
13697,29062384ce4975f78aeba6a496510386,e016365b624d1755f3e2d0e8f1631ebb,249.08
52706,cac62199ea603b63ffb529a162c596c5,e016365b624d1755f3e2d0e8f1631ebb,98.69
58012,a17af8e6044c5ccaa87b1d97559dc554,e016365b624d1755f3e2d0e8f1631ebb,87.80


In [36]:
top_3_payments_bcostumers.head()

,order_id,customer_unique_id,payment_value
0,03caa2c082116e1d31e67e9ae3700499,0a0a92112bd4c708ca5fde585afaa872,13664.08
1,736e1922ae60d0d6a89247b851902527,763c8b1c9c68a0229c42c9fc6f662b93,7274.88
2,0812eb902a67711a1cb742b3cdaa65ae,dc4802a71eae9be1dd28f5d788ceb526,6929.31
3,fefacc66af859508bf1a7934eab1e97f,459bef486812aa25204be022145caa62,6922.21
4,f5136e38d1a14a4dbd87dff67da82701,ff4159b92c40ebe40454e3e6a7c35ed6,6726.66


### Reducir volumen de customers y geolocations
### [BBT3-30](https://alkemy-labs.atlassian.net/browse/BBT3-30) Fabricio

In [37]:
# Utilizo el Dataframe creado anteriormente (BBT3-20), y lo ordeno por 'customer_city' y 'customer_unique_id'
customers_geolocation_sorted = geolocation_customer_df.sort_values(by=['customer_city', 'customer_unique_id'], ascending=[True, True])

In [38]:
customers_geolocation_sorted.head()

,geolocation_zip_code_prefix,mean_lat,mean_lng,std_lat,std_lng,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
19382,38540,-18.474476,-47.407837,0.030707,0.015128,f11eb8f0b8b87510a93e3e1aa10b0ade,64ee476500a01beb94df40f97a108c50,38540,abadia dos dourados,MG
43899,38540,-18.474476,-47.407837,0.030707,0.015128,a23e3f9a2b656b23b7e52075964b42cd,afddf43a03a9941624ed42c0b2c17280,38540,abadia dos dourados,MG
74314,38540,-18.474476,-47.407837,0.030707,0.015128,9e01f714a2b3b8962c222cf2b74c20dc,e1feae9083c4c2895ddf6dc80526a85d,38540,abadia dos dourados,MG
42185,72940,-16.193742,-48.709521,0.004574,0.003569,576d71ddb21b21763cfedce73b902180,8d76b559181609308fcae630ea64cd61,72940,abadiania,GO
47779,35620,-19.158206,-45.446300,0.006980,0.006775,2bf14bf7a1f0610883d264cae6bc4de6,1605eaf5744972f09529a6ce287e9e47,35620,abaete,MG


In [39]:
# Tomar, de las agrupaciones por 'customer_city', los ultimos 30 valores de cada uno
last_30_customer_grouped_by_location = customers_geolocation_sorted.groupby('customer_city').tail(30)

# Tomar las columnas necesarias para el analisis correspondiente
selected_columns = ['customer_zip_code_prefix', 'customer_unique_id', 'mean_lat', 'mean_lng', 'std_lat', 'std_lng', 'customer_city', 'customer_state']
last_30_customer_filtered = last_30_customer_grouped_by_location[selected_columns]

In [40]:
# Corroborar que ahora, como máximo hay 30 clientes de cada ciudad
last_30_customer_filtered.groupby('customer_city')['customer_unique_id'].count().sort_values()

customer_city
ferreiros               1
canaa                   1
morro do ferro          1
santana de pirapama     1
canapi                  1
                       ..
taubate                30
boituva                30
tatui                  30
ouro preto             30
lagoa santa            30
Name: customer_unique_id, Length: 4074, dtype: int64

In [41]:
last_30_customer_filtered.head()

,customer_zip_code_prefix,customer_unique_id,mean_lat,mean_lng,std_lat,std_lng,customer_city,customer_state
19382,38540,64ee476500a01beb94df40f97a108c50,-18.474476,-47.407837,0.030707,0.015128,abadia dos dourados,MG
43899,38540,afddf43a03a9941624ed42c0b2c17280,-18.474476,-47.407837,0.030707,0.015128,abadia dos dourados,MG
74314,38540,e1feae9083c4c2895ddf6dc80526a85d,-18.474476,-47.407837,0.030707,0.015128,abadia dos dourados,MG
42185,72940,8d76b559181609308fcae630ea64cd61,-16.193742,-48.709521,0.004574,0.003569,abadiania,GO
47779,35620,1605eaf5744972f09529a6ce287e9e47,-19.158206,-45.446300,0.006980,0.006775,abaete,MG


### Reducir volumen de orders, payments y customers
### [BBT3-34](https://alkemy-labs.atlassian.net/browse/BBT3-34) Fabricio

Filtrar el DataFrame de órdenes tomando sólo aquellos compras realizadas tarjeta de crédito y cuyo importe sea mayor que el promedio de los pagos, agregando también sus ciudades

In [42]:
query_bbt3_34 = '''
  WITH AvgPayment AS (
    SELECT AVG(payment_value) AS average_payment
    FROM order_payments
)
  SELECT
    c.customer_unique_id,
    p.payment_type,
    p.payment_value,
    c.customer_zip_code_prefix AS zip_code_prefix,
    customer_city,
    customer_state
  FROM orders o
    JOIN order_payments p ON o.order_id = p.order_id
    JOIN customers c ON o.customer_id = c.customer_id
    JOIN AvgPayment ap
  WHERE 1=1
    AND p.payment_type = 'credit_card'
    AND p.payment_value > ap.average_payment
'''
cc_higher_than_prom = pd.read_sql_query(query_bbt3_34, conn)

In [43]:
cc_higher_than_prom.head()

,customer_unique_id,payment_type,payment_value,zip_code_prefix,customer_city,customer_state
0,2ecddf370925aff428e58b4f6021fe38,credit_card,341.09,58410,campina grande,PB
1,a51f8568095ce89a4ffce365dcb7d46d,credit_card,188.73,17501,marilia,SP
2,0ae522661311f598df20272643d39ce6,credit_card,157.45,29300,cachoeiro de itapemirim,ES
3,501598d5821539430a35385474cd44d9,credit_card,244.15,9950,diadema,SP
4,79403f8240aafdd83128106b5b48fd43,credit_card,170.57,68485,pacaja,PA


### [BBT3-31](https://alkemy-labs.atlassian.net/browse/BBT3-31) Pablo

###Filtrar datos, reducir volumen de orders y geolocations

Filtrar el DataFrame de órdenes para resguardar las últimas 30 órdenes de cada ciudad.

In [44]:
# voy a considerar las ultimas 30 ordenes por ciudad ordenando las ordenes segun
# la fecha de compra 'order_purchase_timestamp'
geolocation_orders_sorted_time_df = geolocation_orders_df.sort_values('order_purchase_timestamp', ascending=False)
# chequeo que la cantidad de ordenes por ciudad
geolocation_orders_sorted_time_df.groupby('customer_city').agg({'order_purchase_timestamp': 'count'}).sort_values('order_purchase_timestamp', ascending=False).head(10)

,order_purchase_timestamp
customer_city,
sao paulo,15538
rio de janeiro,6882
belo horizonte,2773
brasilia,1960
curitiba,1521
campinas,1444
porto alegre,1379
salvador,1241
guarulhos,1189


In [45]:
# tomo las primeras 30 filas (ya ordenadas por fecha) para cada ciudad.
orders_last30_by_city_df = geolocation_orders_sorted_time_df.groupby('customer_city').head(30)
# chequeo que la cantidad de ordenes por ciudad
orders_last30_by_city_df.groupby('customer_city').agg({'order_purchase_timestamp': 'count'}).sort_values('order_purchase_timestamp', ascending=False).head(10)

,order_purchase_timestamp
customer_city,
luis eduardo magalhaes,30
rio das ostras,30
coronel fabriciano,30
rio claro,30
rio branco,30
taubate,30
ribeirao preto,30
ribeirao pires,30
telemaco borba,30


In [46]:
cant_registros_originales = len(geolocation_orders_sorted_time_df)
cant_registros_finales = len(orders_last30_by_city_df)
print(f'Cantidad de registros originales:\t {cant_registros_originales}')
print(f'Cantidad de registros finales:\t\t {cant_registros_finales}')
print(f'Dataframe_final: orders_last30_by_city_df')

Cantidad de registros originales:	 99163
Cantidad de registros finales:		 30280
Dataframe_final: orders_last30_by_city_df


### [BBT3-32](https://alkemy-labs.atlassian.net/browse/BBT3-32) Pablo

###Filtrar datos, reducir volumen de orders y reviews

Filtrar el DataFrame de órdenes para resguardar las revisiones con puntaje mayor o igual a 3 y mayor importe de la órden (órdenes con mayor ingreso).

In [47]:
#elimino la columna order_id que se encuentra duplicada
order_reviews_reduced_df = orders_reviews_df.loc[: , ~orders_reviews_df.columns.duplicated()]
# modifico el tipo de datos de la columna score
order_reviews_reduced_df['review_score'] = order_reviews_reduced_df['review_score'].astype(int)
#filtro por score
min_score = 3
orders_reviews__scores_df = order_reviews_reduced_df[orders_reviews_df['review_score'] >= min_score]

<ipython-input-47-7845ad3963ae>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_reviews_reduced_df['review_score'] = order_reviews_reduced_df['review_score'].astype(int)


In [48]:
# para obtener ordenes con mayores ingresos necesito informacion de los pagos que se encuentra
# en el dataframe 'order_payments'.
# Merge con 'order_payments' por columna 'order_id'
orders_reviews__scores_payments_df = orders_reviews__scores_df.merge(order_payments_df, how='inner', on='order_id')

# considero mayor ingreso a aquellas ordenes con un minimo de payment_value de 'min_payment_value'
min_payment_value = 100
filter_orders_score3_and_payments_value_df = orders_reviews__scores_payments_df[orders_reviews__scores_payments_df['payment_value'] >= min_payment_value]
filter_orders_score3_and_payments_value_df.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,payment_type,payment_installments,payment_value
0,73fc7af87114b39712e6da79b0a377eb,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,2018-01-11 15:47:59,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00,7bc2406110b926393aa56f80a40eba40,4,None,None,2018-01-18 00:00:00,2018-01-18 21:46:59,credit_card,8,397.26
2,f9e4b658b201a9f2ecdecbb34bed034b,e226dfed6544df5b7b87a48208690feb,delivered,2018-02-03 09:56:22,2018-02-03 10:33:41,2018-02-06 16:18:28,2018-02-16 17:28:48,2018-03-09 00:00:00,228ce5500dc1d8e020d8d1322874b6f0,5,None,None,2018-02-17 00:00:00,2018-02-18 14:36:24,credit_card,1,194.12
3,658677c97b385a9be170737859d3511b,de6dff97e5f1ba84a3cd9a3bc97df5f6,delivered,2017-04-09 17:41:13,2017-04-09 17:55:19,2017-04-10 14:24:47,2017-04-20 09:08:35,2017-05-10 00:00:00,e64fb393e7b32834bb789ff8bb30750e,5,None,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,credit_card,1,222.84


In [49]:
cant_registros_originales = len(orders_reviews__scores_payments_df)
cant_registros_finales = len(filter_orders_score3_and_payments_value_df)
print(f'Cantidad de registros originales:\t {cant_registros_originales}')
print(f'Cantidad de registros finales:\t\t {cant_registros_finales}')
print(f'Dataframe_final: filter_orders_score3_and_payments_value_df')

Cantidad de registros originales:	 88373
Cantidad de registros finales:		 43367
Dataframe_final: filter_orders_score3_and_payments_value_df


### [BBT3-33](https://alkemy-labs.atlassian.net/browse/BBT3-33) Pablo

###Filtrar datos, reducir volumen de orders_items, products y products_category_translation

Filtrar el DataFrame de las órdenes con mayor o igual importe al promedio de venta y resguardarlos junto con la información de la categoría traducida.  

In [50]:
# calculo precio total para todos los order_items
orderitems_products_df['total_price'] = orderitems_products_df['price'] + orderitems_products_df['freight_value']

#agrupo por order_id y sumo el precio total de todos sus order_items
orders_total_price_df = orderitems_products_df.groupby('order_id').agg({'total_price': 'sum'})
#calculo el promedio de venta de todo el dataset de ordenes
promedio_total_price = orders_total_price_df['total_price'].mean()
print(f'Promedio de venta: {promedio_total_price}')
#filtro aquellas ordenes con precio de venta mayor al promedio
filter_orders_by_mean_df = orderitems_products_df[orderitems_products_df['total_price'] >= promedio_total_price]
filter_orders_by_mean_df.head(3)

Promedio de venta: 160.7087515423213


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name,product_category_name_english,total_price
1,5236307716393b7114b53ee991f36956,1,3aa071139cb16b67ca9e5dea641aaa2f,b561927807645834b59ef0d16ba55a24,2018-02-06 19:11:15,248.0,17.99,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,artes,art,265.99
6,9632facd8bd95315d63a23bf616d85b0,1,732bd381ad09e530fe0a5f457d81becb,8b8cfc8305aa441e4239358c9f6f2485,2018-01-18 12:39:24,958.0,27.76,732bd381ad09e530fe0a5f457d81becb,cool_stuff,56.0,1272.0,4.0,18350.0,70.0,24.0,44.0,cool_stuff,cool_stuff,985.76
7,c6343db6c1801f9c3301166f02931116,1,732bd381ad09e530fe0a5f457d81becb,8b8cfc8305aa441e4239358c9f6f2485,2017-12-11 11:30:51,968.0,44.30,732bd381ad09e530fe0a5f457d81becb,cool_stuff,56.0,1272.0,4.0,18350.0,70.0,24.0,44.0,cool_stuff,cool_stuff,1012.30


In [51]:
cant_registros_originales = len(orderitems_products_df)
cant_registros_finales = len(filter_orders_by_mean_df)
print(f'Cantidad de registros originales:\t {cant_registros_originales}')
print(f'Cantidad de registros finales:\t\t {cant_registros_finales}')
print(f'Dataframe_final: filter_orders_by_mean_df')

Cantidad de registros originales:	 111023
Cantidad de registros finales:		 27085
Dataframe_final: filter_orders_by_mean_df


## Carga de datos a  archivos .csv
### [BBT3-35](https://alkemy-labs.atlassian.net/browse/BBT3-41), [BBT3-36](https://alkemy-labs.atlassian.net/browse/BBT3-41), [BBT3-37](https://alkemy-labs.atlassian.net/browse/BBT3-41) y [BBT3-38](https://alkemy-labs.atlassian.net/browse/BBT3-41) Judith

### [BBT3-39](https://alkemy-labs.atlassian.net/browse/BBT3-41), [BBT3-40](https://alkemy-labs.atlassian.net/browse/BBT3-41) y [BBT3-44](https://alkemy-labs.atlassian.net/browse/BBT3-41) Fabricio
### [BBT3-41](https://alkemy-labs.atlassian.net/browse/BBT3-41), [BBT3-42](https://alkemy-labs.atlassian.net/browse/BBT3-41), [BBT3-43](https://alkemy-labs.atlassian.net/browse/BBT3-41) Pablo

In [52]:
#Creacion de carpeta 'Results' para la carga de datos
nombre_carpeta = 'Results'
#comprobacion de existencia de la carpeta
carpeta_existe = nombre_carpeta in os.listdir('.')

if not carpeta_existe:
  os.mkdir(nombre_carpeta)

In [53]:
# armado de listas con dataframes a cargar y sus correspondientes nombres de archivos
lista_df_analisis = [customers_and_orders_filtrado_df,     # A
                     orders_and_order_items_filtrado_df,   # B
                     items_seller_filtrado_df,             # C
                     items_products_filtrado_df,           # D
                     top_3_payments_bcostumers,            # E
                     last_30_customer_filtered,            # F
                     orders_last30_by_city_df,             # G
                     filter_orders_score3_and_payments_value_df,  # H
                     filter_orders_by_mean_df,               # I
                     cc_higher_than_prom                    # J
                     ]
lista_nombres_archivos = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
lista_nombres_archivos = ['Analysis_'+letra+'.csv' for letra in lista_nombres_archivos]

In [54]:
#Carga de dataframes a archivos
if len(lista_df_analisis) == len(lista_nombres_archivos):
  for i in range(len(lista_nombres_archivos)):
    path = os.path.join(nombre_carpeta, lista_nombres_archivos[i])
    lista_df_analisis[i].to_csv(path, header=True, sep=',')

## Extracción de datos desde CSV a Spark

### [BBT3-45](https://alkemy-labs.atlassian.net/browse/BBT3-45) [BBT3-46](https://alkemy-labs.atlassian.net/browse/BBT3-46) [BBT3-47](https://alkemy-labs.atlassian.net/browse/BBT3-47) [BBT3-48](https://alkemy-labs.atlassian.net/browse/BBT3-48) - Judith
### [BBT3-49](https://alkemy-labs.atlassian.net/browse/BBT3-49) [BBT3-50](https://alkemy-labs.atlassian.net/browse/BBT3-50) [BBT3-54](https://alkemy-labs.atlassian.net/browse/BBT3-54) - Fabricio
### [BBT3-51](https://alkemy-labs.atlassian.net/browse/BBT3-51) [BBT3-52](https://alkemy-labs.atlassian.net/browse/BBT3-52) [BBT3-53](https://alkemy-labs.atlassian.net/browse/BBT3-53) - Pablo

In [55]:
# Crear la sesion de Spark
spark = SparkSession.builder.appName('LecturaCSV').getOrCreate()

In [57]:
# Iteramos sobre cada archivo dentro de la carpeta 'Results' que terminen en .csv y agregar su ruta a una lista
archivos_csv = [os.path.join(nombre_carpeta, archivo) for archivo in os.listdir(nombre_carpeta) if archivo.endswith('.csv')]

# Tomar en dataframes de spark cada uno de los archivos csv
pyspark_dataframes = [spark.read.csv(archivo, header=True, inferSchema=True) for archivo in archivos_csv]

In [60]:
# Mostrar el dataframe en el indice 0
pyspark_dataframes[0].show()

+------+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+----------------------+----------------+------------+
|   _c0|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date|price|freight_value|seller_zip_code_prefix|     seller_city|seller_state|
+------+--------------------+-------------+--------------------+--------------------+-------------------+-----+-------------+----------------------+----------------+------------+
| 27652|3ee6513ae7ea23bdf...|            1|8a3254bee785a526d...|96804ea39d96eb908...|2018-05-04 03:55:26| 0.85|        18.23|                 38950|            ibia|          MG|
| 87081|c5bdd8ef3c0ec4202...|            2|8a3254bee785a526d...|96804ea39d96eb908...|2018-05-07 02:55:22| 0.85|         22.3|                 38950|            ibia|          MG|
| 48625|6e864b3f0ec710311...|            1|8a3254bee785a526d...|96804ea39d96eb908...|2018-05-02 20:30:34|